<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ce9bd03e69d798bbbea891aa26c643bc229b99b80552228d735eb37fe47ee304
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-16 11:10:50
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.18 L
Current:  1.42 C
-------------------
Today PnL: 57.45 K (0.41%)
Current PnL: -16.95 L (-11.4%)
CY Booked + Current PnL: -5.92 L (-3.98%)
-------------------
Total profit:  2.24 L
Total loss:  -19.19 L
-------------------
Total Booked + Current PnL: 21.06 L (17.3%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.75%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.85 L (58.24%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.69%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SBIN,863.00,-11.34,63.0,M-LC,4.22,218916.0,17436.0,9786.0,0.16,8.65,4.47,13.51,60.0,1.78,1.56,22.34,XY25,NTT,BANKS
5,APOLLOHOSP,8285.00,-13.75,57.0,H-LC,11.05,172337.0,16160.0,9927.0,0.22,10.35,5.76,16.71,22.0,1.63,1.23,29.59,X40N,BTT,HEALTHCARE
78,TTKPRESTIG,770.00,104.48,59.0,M-SC,4.42,88124.0,-12653.0,12743.0,0.08,-12.56,14.46,0.09,245.0,-0.99,0.63,15.78,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-42.83,68.0,H-MC,5.66,113998.0,13494.0,17145.0,-0.78,13.43,15.04,30.49,79.0,0.79,0.81,50.18,MH,ATH,METALS
42,ITC,452.00,-37.50,54.0,H-LC,2.86,202250.0,2112.0,18769.0,0.23,1.06,9.28,10.44,4.0,0.11,1.44,6.68,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-29.69,56.0,H-SC,17.30,201175.0,1221.0,96001.0,0.71,0.61,47.72,48.63,115.0,0.01,1.44,12.23,XR,NTT,MISC
35,ICICIPRULI,790.00,-20.77,44.0,H-MC,2.22,136334.0,546.0,42209.0,-0.32,0.40,30.96,31.48,107.0,0.01,0.97,12.57,X40,ATH,INSURANCE
12,BANKINDIA,190.00,-29.19,62.0,H-MC,5.82,181161.0,1353.0,111052.0,-0.17,0.75,61.30,62.52,88.0,0.01,1.29,35.10,XR,NTT,BANKS
52,PGHH,17973.08,-31.21,49.0,H-MC,4.39,199020.0,-1800.0,70572.0,0.14,-0.90,35.46,34.25,80.0,-0.03,1.42,4.25,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-17.18,36.0,H-LC,5.61,298137.0,-17705.0,130375.0,-0.92,-5.61,43.73,35.67,5.0,-0.14,2.13,7.47,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-12.80,48.0,H-LC,8.41,220229.0,-31539.0,84744.0,-1.72,-12.53,38.48,21.13,27.0,-0.37,1.57,18.46,X40,ATH,PAINTS
31,HCLTECH,1943.91,1.08,49.0,H-LC,10.03,230554.0,-11370.0,74630.0,0.14,-4.70,32.37,26.15,8.0,-0.15,1.65,8.95,X40,ATH,IT
44,JIOFIN,387.00,-0.10,50.0,H-LC,13.39,271459.0,4094.0,61350.0,0.10,1.53,22.60,24.48,48.0,0.07,1.94,57.35,XY24,BTT,FINANCE
20,CIPLA,1795.00,-20.12,50.0,H-LC,5.80,213104.0,8604.0,32818.0,0.47,4.21,15.40,20.25,10.0,0.26,1.52,13.31,X40N,BTT,PHARMA


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.20,-2.60,62.0,H-LC,1.01,241650.0,-4501.0,83708.0,0.53,-1.83,34.64,32.18,16.0,-0.05,1.73,33.56,X200,ATH,IT
42,ITC,452.00,-37.50,54.0,H-LC,2.86,202250.0,2112.0,18769.0,0.23,1.06,9.28,10.44,4.0,0.11,1.44,6.68,X40,NTT,FMCG
51,NESTLEIND,1377.00,-8.10,58.0,H-LC,4.14,282695.0,17269.0,39521.0,-0.36,6.51,13.98,21.40,11.0,0.44,2.02,14.06,XY25,NTT,FMCG
57,RELIANCE,1533.00,-12.54,57.0,H-LC,5.59,219289.0,8923.0,19868.0,0.46,4.24,9.06,13.68,37.0,0.45,1.57,21.47,XY25,NTT,REFINERIES
84,VBL,671.64,-17.18,36.0,H-LC,5.61,298137.0,-17705.0,130375.0,-0.92,-5.61,43.73,35.67,5.0,-0.14,2.13,7.47,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,67.49,72.0,H-SC,8.40,145185.0,-37590.0,148655.0,8.16,-20.57,102.39,60.77,125.0,-0.25,1.04,30.04,XR,NTT,JEWELLERY
22,COFFEEDAY,80.00,-39.64,56.0,L-SC,42.01,88186.0,-25363.0,65416.0,3.40,-22.34,74.18,35.27,268.0,-0.39,0.63,115.84,XR,NTT,HOTELS
55,RAJOOENG,143.33,-33.95,45.0,H-SC,10.63,100000.0,-2500.0,43330.0,2.88,-2.44,43.33,39.83,114.0,-0.06,0.71,4.53,AR,ATH,MISC
65,SHALBY,327.00,1086.60,70.0,M-SC,2.30,161282.0,-19861.0,65335.0,2.75,-10.96,40.51,25.10,235.0,-0.30,1.15,29.25,XY24,NTT,HEALTHCARE
8,ATULAUTO,844.00,3916.22,65.0,M-SC,6.45,125139.0,-22229.0,74045.0,2.39,-15.08,59.17,35.16,236.0,-0.30,0.89,27.82,XY24,NTT,AUTO


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,-12.80,48.0,H-LC,8.41,220229.0,-31539.0,84744.0,-1.72,-12.53,38.48,21.13,27.0,-0.37,1.57,18.46,X40,ATH,PAINTS
27,GREENPANEL,537.00,237.21,66.0,M-SC,4.74,150475.0,-32603.0,111577.0,-1.61,-17.81,74.15,43.14,230.0,-0.29,1.07,41.54,XY24,NTT,MISC
76,TITAGARH,1548.00,-0.35,66.0,H-SC,4.16,163546.0,-28469.0,107352.0,-1.45,-14.83,65.64,41.08,158.0,-0.27,1.17,38.59,XY24,NTT,ENGINEERING
4,ANGELONE,3033.00,3.06,35.0,H-SC,9.27,182499.0,-8507.0,66211.0,-1.17,-4.45,36.28,30.21,157.0,-0.13,1.30,15.50,X40N,NTT,FINANCE
33,HONAUT,58357.33,-21.65,46.0,H-SC,2.54,110445.0,-17493.0,64632.0,-1.14,-13.67,58.52,36.84,143.0,-0.27,0.79,13.07,X40N,ATH,ELECTRICAL


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,97.20,58.0,H-SC,12.44,124902.0,-17577.0,37096.0,-0.98,-12.34,29.70,13.70,163.0,-0.47,0.89,47.42,OX40N,NTT,BANKS
78,TTKPRESTIG,770.0,104.48,59.0,M-SC,4.42,88124.0,-12653.0,12743.0,0.08,-12.56,14.46,0.09,245.0,-0.99,0.63,15.78,OX40N,NTT,DURABLES
19,CERA,9475.0,-20.29,45.0,H-SC,2.16,145464.0,-30439.0,72456.0,0.09,-17.30,49.81,23.89,149.0,-0.42,1.04,25.63,OX40N,NTT,CERAMICS
45,KANSAINER,340.0,-67.61,58.0,H-SC,1.74,222876.0,-46791.0,83133.0,0.61,-17.35,37.30,13.47,138.0,-0.56,1.59,12.73,XY24,NTT,PAINTS
73,TATAELXSI,9161.0,-20.18,52.0,H-MC,7.82,108015.0,-23820.0,66040.0,0.00,-18.07,61.14,32.03,98.0,-0.36,0.77,20.99,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4476.75,-26.12,54.0,H-LC,12.83,293910.0,-52046.0,126910.0,0.48,-15.04,43.18,21.64,1.0,-0.41,2.10,4.12,X40,ATH,IT
40,INFY,2275.00,-17.80,52.0,H-LC,8.82,320395.0,7507.0,164170.0,-0.28,2.40,51.24,54.87,3.0,0.05,2.29,9.21,X40,BTT,IT
42,ITC,452.00,-37.50,54.0,H-LC,2.86,202250.0,2112.0,18769.0,0.23,1.06,9.28,10.44,4.0,0.11,1.44,6.68,X40,NTT,FMCG
84,VBL,671.64,-17.18,36.0,H-LC,5.61,298137.0,-17705.0,130375.0,-0.92,-5.61,43.73,35.67,5.0,-0.14,2.13,7.47,X40N,ATH,FMCG
1,ABB,7934.00,-38.60,63.0,H-LC,11.13,256723.0,-4896.0,124100.0,0.18,-1.87,48.34,45.57,7.0,-0.04,1.83,9.24,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,65.0,M-MC,6.96,228630.0,3668.0,163996.0,-0.02,1.63,71.73,74.53,192.0,0.02,1.63,34.41,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,11.62,60.0,H-MC,3.70,192770.0,28100.0,39865.0,2.04,17.06,20.68,41.28,89.0,0.70,1.38,26.41,X40N,ATH,AC
85,VOLTAS,1918.49,2.00,61.0,H-MC,2.67,213960.0,22218.0,73816.0,0.75,11.59,34.50,50.08,99.0,0.30,1.53,19.17,XY25,ATH,AC
30,HAVELLS,2069.16,1.19,65.0,H-MC,1.82,250988.0,3257.0,71808.0,0.86,1.31,28.61,30.30,92.0,0.05,1.79,14.78,X40,ATH,ELECTRICAL
24,DABUR,735.00,-3.12,57.0,H-MC,6.57,210892.0,13864.0,75752.0,-0.03,7.04,35.92,45.49,102.0,0.18,1.51,20.99,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-42.83,68.0,H-MC,5.66,113998.0,13494.0,17145.0,-0.78,13.43,15.04,30.49,79.0,0.79,0.81,50.18,MH,ATH,METALS
86,WHIRLPOOL,2270.00,-39.15,59.0,M-SC,0.62,102825.0,11326.0,67422.0,2.04,12.38,65.57,86.07,223.0,0.17,0.73,49.32,XR,NTT,DURABLES
37,INDIAMART,4850.92,-50.72,52.0,H-SC,12.20,137752.0,14416.0,119348.0,-0.38,11.69,86.64,108.45,119.0,0.12,0.98,36.65,AR,ATH,MISC
87,WIPRO,420.00,-11.52,53.0,M-LC,5.61,155763.0,4818.0,104642.0,-0.02,3.19,67.18,72.51,53.0,0.05,1.11,9.41,XR,NTT,IT
36,IEX,219.00,-29.69,56.0,H-SC,17.30,201175.0,1221.0,96001.0,0.71,0.61,47.72,48.63,115.0,0.01,1.44,12.23,XR,NTT,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ALKYLAMINE,4546.37,-4.51,47.0,H-SC,13.33,88808.0,-12155.0,106685.0,0.19,-12.04,120.13,93.63,148.0,-0.11,0.63,32.28,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,56.0,L-SC,42.01,88186.0,-25363.0,65416.0,3.40,-22.34,74.18,35.27,268.0,-0.39,0.63,115.84,XR,NTT,HOTELS
49,MASFIN,398.61,-15.99,57.0,H-SC,12.92,95475.0,-2505.0,24107.0,-0.52,-2.56,25.25,22.05,152.0,-0.10,0.68,38.78,XR,ATH,FINANCE
50,NATIONALUM,247.44,-42.83,68.0,H-MC,5.66,113998.0,13494.0,17145.0,-0.78,13.43,15.04,30.49,79.0,0.79,0.81,50.18,MH,ATH,METALS
46,KPIGREEN,731.05,3.26,46.0,H-SC,10.25,123228.0,-2069.0,60998.0,1.76,-1.65,49.50,47.03,141.0,-0.03,0.88,54.29,MH,ATH,POWER


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-39.64,56.0,L-SC,42.01,88186.0,-25363.0,65416.0,3.40,-22.34,74.18,35.27,268.0,-0.39,0.63,115.84,XR,NTT,HOTELS
8,ATULAUTO,844.0,3916.22,65.0,M-SC,6.45,125139.0,-22229.0,74045.0,2.39,-15.08,59.17,35.16,236.0,-0.30,0.89,27.82,XY24,NTT,AUTO
82,VAIBHAVGBL,521.0,67.49,72.0,H-SC,8.40,145185.0,-37590.0,148655.0,8.16,-20.57,102.39,60.77,125.0,-0.25,1.04,30.04,XR,NTT,JEWELLERY
65,SHALBY,327.0,1086.60,70.0,M-SC,2.30,161282.0,-19861.0,65335.0,2.75,-10.96,40.51,25.10,235.0,-0.30,1.15,29.25,XY24,NTT,HEALTHCARE
27,GREENPANEL,537.0,237.21,66.0,M-SC,4.74,150475.0,-32603.0,111577.0,-1.61,-17.81,74.15,43.14,230.0,-0.29,1.07,41.54,XY24,NTT,MISC


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.46
1,25,42.77
2,50,73.21


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.27
LC    33.94
MC    22.78
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.53
X40      14.99
X40N     12.77
XR       11.40
XY25      9.63
AR        8.51
OX40N     8.05
X200      1.73
MH        1.69
X5K       1.43
SR        1.26
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.39
H-LC    25.84
H-MC    19.87
M-SC    13.35
M-LC     7.05
M-MC     2.61
L-SC     1.53
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.13,-14.44,76.83
FMCG,11.61,-4.15,40.59
FINANCE,9.48,-15.88,61.27
BANKS,7.61,-12.24,63.92
MISC,7.38,-11.06,72.82
PAINTS,5.77,-14.31,31.35
ELECTRICAL,5.60,-4.94,62.78
AC,3.61,4.03,28.47
INSURANCE,3.35,-7.27,44.06


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2848877.0,22
XR,1311358.0,14
AR,1302352.0,9
X40,748696.0,10
X40N,624551.0,10
OX40N,520292.0,10
XY25,403751.0,7
SR,244958.0,2
X5K,117982.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3435692.0,29
M-SC,1372172.0,17
H-LC,1257873.0,15
H-MC,1171489.0,15
M-LC,390968.0,5
M-MC,312373.0,2
L-SC,246064.0,3
L-MC,60239.0,1
L-LC,37798.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1039673.0      6
M-SC       XY24       792337.0      7
H-SC       AR         784020.0      5
           XR         760148.0      7
H-MC       XY24       484586.0      4
H-LC       X40        469223.0      5
           AR         356116.0      2
M-MC       XY24       312373.0      2
H-SC       X40N       297507.0      4
           OX40N      248388.0      4
           SR         244958.0      2
H-LC       X40N       228087.0      4
H-MC       X40        210997.0      4
           XY25       182712.0      2
L-SC       XR         162459.0      2
M-SC       AR         162216.0      2
M-LC       XY24       158558.0      2
M-SC       OX40N      122259.0      4
M-LC       X5K        117982.0      1
M-SC       XY25       114066.0      1
           XR         112818.0      2
H-MC       XR         111052.0      1
M-LC       XR         104642.0      1
H-MC       X40N        98957.0      2
H-LC       X200        83708.0      1
L-SC       OX40N       83605.0      1
M-SC       X40         68476.0      1
H-MC       OX40N       66040.0      1
H-LC       XY24        61350.0      1
H-SC       MH          60998.0      1
L-MC       XR          60239.0      1
H-LC       XY25        59389.0      2
L-LC       XY25        37798.0      1
H-MC       MH          17145.0      1
M-LC       XY25         9786.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
